In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from hyperopt import hp
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe

## load data

We retrieve the data from the different CSV files, then we store them in corresponding variables (training set in X_train)

In [3]:
df_train = pd.read_csv("../data/archive/train.csv")
df_train
X_train, y_train = df_train["review"],  df_train["polarity"]

In [4]:
df_test = pd.read_csv("../data/archive/test.csv")
X_test, y_test = df_test["review"],  df_test["polarity"]
df_val = pd.read_csv("../data/archive/valid.csv")
X_val, y_val= df_val["review"],  df_val["polarity"]

We check for the presence of missing data

In [5]:
df_train.isna().sum(), df_train.isnull().sum()

(Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64,
 Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64)

checking the balance of labels

In [6]:
np.unique(df_train["polarity"], return_counts=True)

(array([0, 1]), array([79413, 80587]))

it's balance

## preprocessing

In [7]:
STOP_WORDS = list(
    """
a à â abord afin ah ai aie ainsi ait allaient allons
alors anterieur anterieure anterieures antérieur antérieure antérieures
apres après as assez attendu au
aupres auquel aura auraient aurait auront
aussi autre autrement autres autrui aux auxquelles auxquels avaient
avais avait avant avec avoir avons ayant

bas basee bat

c' c’ ça car ce ceci cela celle celle-ci celle-la celle-là celles celles-ci celles-la celles-là
celui celui-ci celui-la celui-là cent cependant certain certaine certaines certains certes ces
cet cette ceux ceux-ci ceux-là chacun chacune chaque chez ci cinq cinquantaine cinquante
cinquantième cinquième combien comme comment compris concernant

d' d’ da dans de debout dedans dehors deja dejà delà depuis derriere
derrière des desormais desquelles desquels dessous dessus deux deuxième
deuxièmement devant devers devra different differente differentes differents différent
différente différentes différents dire directe directement dit dite dits divers
diverse diverses dix dix-huit dix-neuf dix-sept dixième doit doivent donc dont
douze douzième du duquel durant dès déja déjà désormais

effet egalement eh elle elle-meme elle-même elles elles-memes elles-mêmes en encore
enfin entre envers environ es ès est et etaient étaient etais étais etait était
etant étant etc etre être eu eux eux-mêmes exactement excepté également

fais faisaient faisant fait facon façon feront font

gens

ha hem hep hi ho hormis hors hou houp hue hui huit huitième
hé i il ils importe

j' j’ je jusqu jusque juste

l' l’ la laisser laquelle le lequel les lesquelles lesquels leur leurs longtemps
lors lorsque lui lui-meme lui-même là lès

m' m’ ma maint maintenant mais malgre malgré me meme memes merci mes mien
mienne miennes miens mille moi moi-meme moi-même moindres moins
mon même mêmes

n' n’ na ne neanmoins neuvième ni nombreuses nombreux nos notamment
notre nous nous-mêmes nouveau nul néanmoins nôtre nôtres

o ô on ont onze onzième or ou ouias ouste outre
ouvert ouverte ouverts où

par parce parfois parle parlent parler parmi partant
pas pendant pense permet personne peu peut peuvent peux plus
plusieurs plutot plutôt possible possibles pour pourquoi
pourrais pourrait pouvait prealable precisement
premier première premièrement
pres procedant proche près préalable précisement pu puis puisque

qu' qu’ quand quant quant-à-soi quarante quatorze quatre quatre-vingt
quatrième quatrièmement que quel quelconque quelle quelles quelqu'un quelque
quelques quels qui quiconque quinze quoi quoique

relative relativement rend rendre restant reste
restent retour revoici revoila revoilà

s' s’ sa sait sans sauf se seize selon semblable semblaient
semble semblent sent sept septième sera seraient serait seront ses seul seule
seulement seuls seules si sien sienne siennes siens sinon six sixième soi soi-meme soi-même soit
soixante son sont sous souvent specifique specifiques spécifique spécifiques stop
suffisant suffisante suffit suis suit suivant suivante
suivantes suivants suivre sur surtout

t' t’ ta tant te tel telle tellement telles tels tenant tend tenir tente
tes tien tienne tiennes tiens toi toi-meme toi-même ton touchant toujours tous
tout toute toutes treize trente tres trois troisième troisièmement très
tu té

un une unes uns

va vais vas vers via vingt voici voila voilà vont vos
votre votres vous vous-mêmes vu vé vôtre vôtres

y

""".split()
)

source https://github.com/explosion/spaCy/blob/master/spacy/lang/fr/stop_words.py


we use  TfidfVectorizer, it use count vectorizer followed bytransform tfid
- count vectorizer create a matrice document x word  the cell i,j correspond of ith document avec jth word and it count the  number of occurence of the word j in i
- tfid use this matrix to calculate the tfidf matrix

### regression lineaire

In [8]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42))
    ])
    
pipeline.fit(X_train, y_train)
pipeline

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000,
                                 stop_words=['a', 'à', 'â', 'abord', 'afin',
                                             'ah', 'ai', 'aie', 'ainsi', 'ait',
                                             'allaient', 'allons', 'alors',
                                             'anterieur', 'anterieure',
                                             'anterieures', 'antérieur',
                                             'antérieure', 'antérieures',
                                             'apres', 'après', 'as', 'assez',
                                             'attendu', 'au', 'aupres',
                                             'auquel', 'aura', 'auraient',
                                             'aurait', ...])),
                ('classifier', LogisticRegression(random_state=42))])

In [9]:
pipeline.score(X_test, y_test)

0.9198625

In [10]:
# create a pipeline combining TF-IDF vectorization and Logistic Regression
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, penalty='l2'))
    ])
    
# fit the pipeline on the training data
pipeline.fit(X_train, y_train)
# evaluate the pipeline's performance on the validation set
pipeline.score(X_val, y_val)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


0.9326625

In [14]:
# create a pipeline combining TF-IDF vectorization and Logistic Regression with different parameters
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, penalty='l1', solver="liblinear"))
    ])
    
pipeline.fit(X_train, y_train)
pipeline.score(X_val, y_val)

0.92839375

In [15]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, C = 1.4))
    ])
    
pipeline.fit(X_train, y_train)
pipeline.score(X_val, y_val)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


0.93449375

In [17]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, C = 0.1))
    ])
    
pipeline.fit(X_train, y_train)
pipeline.score(X_val, y_val)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


0.9090125

In [ ]:
#avce plus de features le modele est plus precis 

Que signifient precision, recall, accuracy dans le contexte de ce TP ? Qu'en concluez vous sur la métrique à privilégier ?
le rappelle est  la proportion de tous les positifs réels qui ont été correctement classés comme positifs
Précision est la proportion de toutes les classifications positives du modèle qui sont réellement positives. 
l'accuracy  permet de décrire la performance du modèle sur les individus positifs et négatifs de façon symétrique. on prend le nombre de bonne classification divisé sur le total

Ici j'ai choisi l'accuracie puisque c'est équilibré

## Expérimentation de différents modèles et hyperparamètres

In [11]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, stop_words = STOP_WORDS)),
    ('classifier', MLPClassifier())
    ])
    
pipeline.fit(X_train, y_train)
print(pipeline)
pipeline.score(X_test, y_test)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000,
                                 stop_words=['a', 'à', 'â', 'abord', 'afin',
                                             'ah', 'ai', 'aie', 'ainsi', 'ait',
                                             'allaient', 'allons', 'alors',
                                             'anterieur', 'anterieure',
                                             'anterieures', 'antérieur',
                                             'antérieure', 'antérieures',
                                             'apres', 'après', 'as', 'assez',
                                             'attendu', 'au', 'aupres',
                                             'auquel', 'aura', 'auraient',
                                             'aurait', ...])),
                ('classifier', MLPClassifier())])


0.9992375

In [12]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, stop_words = STOP_WORDS)),
    ('classifier', RandomForestClassifier())
    ])
    
pipeline.fit(X_train, y_train)
print(pipeline)
pipeline.score(X_test, y_test)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


KeyboardInterrupt: 

## Hyperopt

In [ ]:
#  optimisation des hyperparametres  pour la regression logistic
def objective(params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=params['tfidf__max_features'])),
        ('logreg', LogisticRegression(C=params['logreg__C']))
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return {'loss': 1 - accuracy, 'status': "ok"}

space = {
    'tfidf__max_features': hp.choice('tfidf__max_features', [1000, 5000, 10000]),
    'logreg__C': hp.loguniform('logreg__C', np.log(0.001), np.log(100))
}

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    return_argmin=False
)


In [ ]:
#  optimisation des hyperparametres  pour le mlp
def objective(params):
    pipeline =  Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ("mlp",MLPClassifier(
        hidden_layer_sizes=params['mlp_hidden_layer_sizes'],
       # activation=params['mlp_activation'],
        solver=params['mlp_solver'],
       # alpha=params['mlp_alpha'],
       # learning_rate=params['mlp_learning_rate'],
        max_iter=1000 )
        )])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return {'loss': 1 - accuracy, 'status': "ok"}

space = {'mlp_hidden_layer_sizes': hp.choice('hidden_layer_sizes', [
        (50,), (100,), (50, 50), (100, 50), (100, 100)
    ]),
   # 'mlp_activation': hp.choice('activation', ['relu', 'tanh']),
    'mlp_solver': hp.choice('solver', ['adam', 'sgd']),
    #'mlp_alpha': hp.loguniform('alpha', np.log(1e-5), np.log(1)),
    #'mlp_learning_rate': hp.choice('learning_rate', ['constant', 'invscaling', 'adaptive'])
}

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    return_argmin=False
)

best
